In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import nltk
#nltk.download()
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
import re
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix

# Data Loading

In [ ]:
sms=pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv')
sms.head()

# EDA

In [ ]:
sms.info()

In [ ]:
sms.describe()

In [ ]:
sms.isnull().sum()

In [ ]:
sms=sms.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
sms.isnull().sum()

In [ ]:
sms=sms.rename({'v1':'label','v2':'text'},axis=1)
sms.head()

In [ ]:
sns.countplot(sms['label'])

In [ ]:
sms['length']=sms['text'].apply(lambda x: len(x))
sns.distplot(sms['length'], kde=True)

In [ ]:
sns.boxplot(y='length', x='label', data=sms)

In [ ]:
dt=sms.groupby('label').mean()
#sns.barplot(y='length', x='label', data=dt)
dt

In [ ]:
sms.groupby('label').describe()

In [ ]:
sms.hist(column='length', by='label', bins=50,figsize=(20,8))

# Data Preprocessing

In [ ]:
ps=PorterStemmer()
def preprocess(x):
    x = x.lower()
    x = re.sub(r'[^0-9a-zA-Z]', ' ', x)
    x = re.sub(r'\s+', ' ', x)
    l = [item for item in x if item not in string.punctuation]
    processed=[ps.stem(i) for i in l if i not in stopwords.words('english')]
    return processed


In [ ]:
X=sms['text']
y=sms['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

# Pipeline 

In [ ]:
def model(model_name,X_train,y_train,X_test,y_test):
    pipeline=Pipeline([
    ('bow', CountVectorizer(analyzer=preprocess)),
    ('tfidf', TfidfTransformer()),
    ('model', model_name),
    ])
    pipeline.fit(X_train,y_train)

    preds=pipeline.predict(X_test)

    print (classification_report(y_test,preds))
    print (confusion_matrix(y_test,preds))
    print('Accuracy:', pipeline.score(X_test, y_test)*100)
    print("Training Score:",pipeline.score(X_train,y_train)*100)
    from sklearn.metrics import accuracy_score
    score = accuracy_score(y_test,preds)
    print(score)


MultinomialNB 

In [ ]:
model_name=MultinomialNB()
model(model_name,X_train,y_train,X_test,y_test)

RF

In [ ]:
model_name=RandomForestClassifier()
model(model_name,X_train,y_train,X_test,y_test)

Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model_name=GradientBoostingClassifier()
model(model_name,X_train,y_train,X_test,y_test)

XGBoost

In [ ]:
model_name=XGBClassifier()
model(model_name,X_train,y_train,X_test,y_test)

SVC

In [ ]:
model_name=SVC()
model(model_name,X_train,y_train,X_test,y_test)